# Sentiment Analysis of Candidate Twitter Accounts
## Bio Sentiment Analysis

### Katon Minhas

In [1]:
import pandas as pd
import numpy as np
#import twint
import nest_asyncio
nest_asyncio.apply()
import re


C:\Users\katon\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### Read Data

In [2]:
path = "C:/Users/Katon/Documents/JHU/SocialMediaAnalytics/final_project/data/"

data = pd.read_csv(path+"user_data.csv")
data.head()

,State,Branch,Type,Party,Candidate,Win,Votes,Incumbent,Handle,Bio,Following,Followers
0,Iowa,Senate,Primary,D,Abby Finkenauer,0,62358,0,Abby4Iowa,['Former U.S. Congresswoman & Iowa State Repre...,671,54113
1,Nevada,Senate,General,R,Adam Laxalt,0,476676,0,AdamLaxalt,None,1450,89610
2,Nevada,Senate,Primary,R,Adam Laxalt,1,97830,0,AdamLaxalt,None,1450,89610
3,Wisconsin,Senate,Primary,D,Alex Lasry,0,40820,0,AlexLasryWI,['Husband to Lauren & dad to Eleanor. Fmr. Can...,215,23344
4,California,Senate,General,D,Alex Padilla,1,3202616,1,AlexPadilla4CA,['U.S. Senator for California. Husband & Dad. ...,1569,112309


#### Clean Data

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

C:\Users\katon\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
# Remove symbols and stopwords from bio, add bio word count column
data['bio_length'] = 0

for i in range(data.shape[0]):
    bio = data.iloc[i, 9]
    if bio != 'None':
        bio_list = [word for word in word_tokenize(bio) if word.isalnum()]
        data.iloc[i, 12] = len(bio_list) # Bio_Length column
        bio_string = ' '.join([word for word in bio_list if not word in stopwords.words()])
        data.iloc[i, 9] = bio_string

In [5]:
data.head()

,State,Branch,Type,Party,Candidate,Win,Votes,Incumbent,Handle,Bio,Following,Followers,Bio_Length
0,Iowa,Senate,Primary,D,Abby Finkenauer,0,62358,0,Abby4Iowa,Congresswoman Iowa State Representative Buildi...,671,54113,15
1,Nevada,Senate,General,R,Adam Laxalt,0,476676,0,AdamLaxalt,None,1450,89610,0
2,Nevada,Senate,Primary,R,Adam Laxalt,1,97830,0,AdamLaxalt,None,1450,89610,0
3,Wisconsin,Senate,Primary,D,Alex Lasry,0,40820,0,AlexLasryWI,Lauren dad Eleanor Fmr Candidate Senate SVP Mi...,215,23344,20
4,California,Senate,General,D,Alex Padilla,1,3202616,1,AlexPadilla4CA,Senator California Husband Dad Former CA Secre...,1569,112309,19


### Bio Sentiment

In [14]:
from textblob import TextBlob
from profanity_check import predict, predict_prob

In [24]:
# add bio sentiment analysis measures
data['bio_polarity'] = 0.0     # col 13
data['bio_subjectivity'] = 0.0 # col 14
data['bio_profanity'] = 0.0    # col 15

for i in range(data.shape[0]):
    bio = data.iloc[i, 9]
    if bio != 'None':
        blob = TextBlob(bio)
        data.iloc[i, 13] = blob.sentiment.polarity
        data.iloc[i, 14] = blob.sentiment.subjectivity
        data.iloc[i, 15] = predict_prob([bio])
    else:
        data.iloc[i, 13] = np.nan
        data.iloc[i, 14] = np.nan
        data.iloc[i, 15] = np.nan

In [26]:
data.head(100)

,State,Branch,Type,Party,Candidate,Win,Votes,Incumbent,Handle,Bio,Following,Followers,Bio_Length,bio_polarity,bio_subjectivity,bio_profanity
0,Iowa,Senate,Primary,D,Abby Finkenauer,0,62358,0,Abby4Iowa,Congresswoman Iowa State Representative Buildi...,671,54113,15,0.0,0.3,0.043760
1,Nevada,Senate,General,R,Adam Laxalt,0,476676,0,AdamLaxalt,None,1450,89610,0,NaN,NaN,NaN
2,Nevada,Senate,Primary,R,Adam Laxalt,1,97830,0,AdamLaxalt,None,1450,89610,0,NaN,NaN,NaN
3,Wisconsin,Senate,Primary,D,Alex Lasry,0,40820,0,AlexLasryWI,Lauren dad Eleanor Fmr Candidate Senate SVP Mi...,215,23344,20,0.0,0.0,0.095739
4,California,Senate,General,D,Alex Padilla,1,3202616,1,AlexPadilla4CA,Senator California Husband Dad Former CA Secre...,1569,112309,19,0.0,0.0,0.100305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Alabama,Senate,Primary,R,Jake Schafer,0,6951,0,NO_HANDLE,None,0,0,0,NaN,NaN,NaN
96,North Carolina,Senate,Primary,D,James Carr Jr,0,21903,0,NO_HANDLE,None,0,0,0,NaN,NaN,NaN
97,Oklahoma,Senate,General,R,James Lankford,1,739302,1,SenatorLankford,Christ husband Cindy father two daughters serv...,749,121439,19,0.0,0.0,0.111747
98,Oklahoma,Senate,Primary,R,James Lankford,1,242984,1,SenatorLankford,Christ husband Cindy father two daughters serv...,749,121439,19,0.0,0.0,0.111747


In [62]:
data.to_csv("C:/Users/Katon/Documents/JHU/SocialMediaAnalytics/final_project/data/user_data_bio_added.csv")